<a href="https://colab.research.google.com/github/madhusudhanrao-ppm/devrel-colab/blob/main/RAGChatbotwithAgentDevelopmentKit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the required packages

In [ ]:
!pip install "oci[adk]" oci langchain-oracledb langchain

Save the OCI config file in Google Drive. the file format should be as shown below
```
[DEFAULT]

user=ocid1.user.oc1..aaaaaaaarvvpvxiwapsybxoefovyd7wpl4q

fingerprint=ca:bf:b7:53:57:78:9b:ff:bb:47:3e:ae:a2:a2:b8:03

tenancy=ocid1.tenancy.oc1..aaaaaaaaj4kymyo4xwxyv3gfa

region=us-phoenix-1

key_file=/root/.oci/yourkeyfile.pem
```


In [2]:
# Create OCI configuration directory if it doesn't exist
import os
oci_dir = '/root/.oci'
if not os.path.exists(oci_dir):
    os.makedirs(oci_dir)

# Mount Google Drive to access OCI configuration files stored in the cloud
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Copy OCI authentication files from Google Drive to the Colab environment
# This includes the OCI config file with credentials and API key
!cp /content/drive/MyDrive/OCI_KEY/config ~/.oci/config
# Copy private key file required for OCI API authentication
!cp /content/drive/MyDrive/OCI_KEY/yourtenancy_sso.pem $oci_dir/yourtenancy_sso.pem

In [4]:
# Import required libraries
import json
import os
from typing import Dict, List, Optional, Any
from datetime import datetime
# Oracle ADK for building AI agents
from oci.addons.adk import Agent, AgentClient, tool
# OCI Object Storage client for managing cloud storage
from oci.object_storage import ObjectStorageClient
# OCI SDK for cloud infrastructure interaction
import oci
# Asynchronous programming support
import asyncio
import nest_asyncio

In [5]:
# ============================================================================
# DOCUMENT STORE - In-memory knowledge base for RAG
# ============================================================================

class DocumentStore:
    """
    Simple in-memory document store for RAG (Retrieval-Augmented Generation).
    Stores documents that can be retrieved and used to augment agent responses.
    """

    def __init__(self):
        """Initialize the document store with sample documents about OCI and RAG."""
        self.documents = {
            "oracle_cloud_overview": {
                "title": "Oracle Cloud Infrastructure Overview",
                "content": """Oracle Cloud Infrastructure (OCI) is a secure, economical and high-performing
                cloud platform. OCI provides a complete suite of cloud computing services including compute,
                storage, networking, database, analytics, and AI services."""
            },
            "adk_intro": {
                "title": "Oracle Agent Development Toolkit",
                "content": """The Oracle Agent Development Toolkit (ADK) enables developers to create
                intelligent agents that can use tools, reason about tasks, and interact with various services.
                Agents can be deployed as endpoints and invoked via API."""
            },
            "rag_basics": {
                "title": "RAG (Retrieval-Augmented Generation) Basics",
                "content": """RAG combines document retrieval with generative AI. When a user asks a question,
                the system retrieves relevant documents from a knowledge base and uses them to generate
                accurate, contextual responses."""
            }
        }

    def search_documents(self, query: str) -> List[Dict[str, str]]:
        """
        Search documents using simple keyword matching.

        Args:
            query (str): Search query to find relevant documents

        Returns:
            List[Dict[str, str]]: List of matching documents with preview text
        """
        query_lower = query.lower()
        results = []

        # Iterate through all documents and find matches
        for doc_id, doc in self.documents.items():
            # Check if query matches in title or content
            title_match = query_lower in doc["title"].lower()
            content_match = query_lower in doc["content"].lower()

            # Add to results if either title or content matches
            if title_match or content_match:
                results.append({
                    "id": doc_id,
                    "title": doc["title"],
                    "preview": doc["content"][:200] + "..."
                })

        return results[:3]  # Return top 3 results


# Global document store instance - used by tool functions
doc_store = DocumentStore()


# ============================================================================
# TOOL DEFINITIONS FOR THE AGENT
# These functions are exposed to the agent as tools it can call to perform tasks
# ============================================================================

@tool
def retrieve_documents(query: str) -> Dict[str, Any]:
    """
    Retrieve relevant documents from the knowledge base based on the query.
    This tool allows the agent to search for and retrieve documents.

    Args:
        query(str): The search query to find relevant documents

    Returns:
        dict: Retrieved documents with metadata, status, and timestamp
    """
    results = doc_store.search_documents(query)

    # Return error if no documents found
    if not results:
        return {
            "status": "not_found",
            "message": f"No documents found for query: '{query}'",
            "documents": []
        }

    # Return successful results with timestamp
    return {
        "status": "success",
        "message": f"Found {len(results)} relevant document(s)",
        "documents": results,
        "timestamp": datetime.now().isoformat()
    }


@tool
def get_document_content(document_id: str) -> Dict[str, Any]:
    """
    Retrieve the full content of a specific document by its ID.
    This tool allows the agent to access complete document text.

    Args:
        document_id(str): The ID of the document to retrieve

    Returns:
        dict: Full document content with metadata or error message
    """
    # Check if document exists in store
    if document_id not in doc_store.documents:
        return {
            "status": "error",
            "message": f"Document not found: {document_id}"
        }

    # Return complete document with metadata
    doc = doc_store.documents[document_id]
    return {
        "status": "success",
        "id": document_id,
        "title": doc["title"],
        "content": doc["content"],
        "retrieved_at": datetime.now().isoformat()
    }


@tool
def search_oci_knowledge_base(topic: str, category: str = "all") -> Dict[str, Any]:
    """
    Search the OCI knowledge base for information about specific topics.
    The agent can use this to find OCI-related information across categories.

    Args:
        topic(str): The topic to search for (e.g., "storage", "compute")
        category(str): Category filter (all, compute, storage, database, ai)

    Returns:
        dict: Search results with relevant OCI information and count
    """
    # OCI knowledge base organized by service category
    knowledge_base = {
        "compute": [
            "VM instances", "Container instances", "Functions",
            "Kubernetes Engine", "Bare Metal Compute"
        ],
        "storage": [
            "Object Storage", "Block Volumes", "File Storage",
            "Archive Storage", "Data Transfer"
        ],
        "database": [
            "Autonomous Database", "MySQL Database", "PostgreSQL Database",
            "NoSQL Database", "Database Backup"
        ],
        "ai": [
            "AI Services", "Generative AI", "Document Understanding",
            "Anomaly Detection", "Data Science"
        ]
    }

    results = []
    topic_lower = topic.lower()

    # Determine which categories to search
    if category == "all":
        categories = knowledge_base.keys()
    else:
        categories = [category] if category in knowledge_base else []

    # Search for topic matches across selected categories
    for cat in categories:
        for item in knowledge_base[cat]:
            if topic_lower in item.lower():
                results.append({"category": cat, "item": item})

    # Return results with metadata
    return {
        "topic": topic,
        "category": category,
        "results": results,
        "count": len(results)
    }


@tool
def get_conversation_context(session_id: str) -> Dict[str, Any]:
    """
    Retrieve conversation context information for maintaining conversation state.
    This helps the agent understand the current conversation session.

    Args:
        session_id(str): The session identifier

    Returns:
        dict: Conversation context with session info, topics, and status
    """
    # Return context structure with session information
    return {
        "session_id": session_id,
        "start_time": datetime.now().isoformat(),
        "context": {
            "user_intent": "general_inquiry",
            "topics_discussed": [],
            "documents_referenced": []
        },
        "status": "active"
    }


# ============================================================================
# RAG CHATBOT CLASS - Main conversational interface
# ============================================================================

class RAGChatbot:
    """
    RAG-enabled conversational AI chatbot using Oracle ADK.
    Combines document retrieval with generative AI to provide accurate, context-aware responses.
    """

    def __init__(
        self,
        agent_endpoint_id: str,
        auth_type: str = "api_key",
        profile: str = "DEFAULT",
        region: str = "us-chicago-1"
    ):
        """
        Initialize the RAG chatbot with OCI authentication and configuration.

        Args:
            agent_endpoint_id (str): The OCID of the deployed agent endpoint
            auth_type (str): Authentication type (api_key, instance_principal, resource_principal)
            profile (str): OCI config profile name to use for authentication
            region (str): OCI region where the agent endpoint is deployed
        """
        self.agent_endpoint_id = agent_endpoint_id
        self.auth_type = auth_type
        self.profile = profile
        self.region = region
        # Store conversation history for context and logging
        self.conversation_history = []
        self.agent = None
        self.client = None

        # Initialize the OCI agent client connection
        self._initialize_client()

    def _initialize_client(self):
        """
        Initialize the OCI Agent Client for communication with the agent endpoint.
        Handles authentication and connection setup.
        """
        try:
            # Create agent client with specified authentication and region
            self.client = AgentClient(
                auth_type=self.auth_type,
                profile=self.profile,
                region=self.region
            )
            print(f"✓ Agent client initialized")
            print(f"  - Region: {self.region}")
            print(f"  - Auth Type: {self.auth_type}")
        except Exception as e:
            print(f"✗ Failed to initialize agent client: {e}")
            raise

    def setup_agent(self, instructions: str = None):
        """
        Set up the agent with instructions and register available tools.
        The agent uses these tools to answer user queries.

        Args:
            instructions (str): Custom system instructions for the agent behavior.
                                If None, uses default RAG instructions.
        """
        # Use default instructions if not provided
        if instructions is None:
            instructions = """You are a helpful RAG-powered assistant for Oracle Cloud Infrastructure.
            Your role is to:
            1. Answer questions about OCI services and features
            2. Retrieve relevant documents from the knowledge base when needed
            3. Provide accurate, context-aware responses based on retrieved information
            4. Guide users through OCI concepts and best practices

            When answering questions:
            - Always search for relevant documents first
            - Cite the documents you used
            - Provide clear, structured answers
            - Ask clarifying questions if needed"""

        try:
            # Create agent with client, endpoint, instructions, and tools
            self.agent = Agent(
                client=self.client,
                agent_endpoint_id=self.agent_endpoint_id,
                instructions=instructions,
                # Register all available tools for the agent to use
                tools=[
                    retrieve_documents,
                    get_document_content,
                    search_oci_knowledge_base,
                    get_conversation_context
                ]
            )

            # Sync instructions and tools to remote agent endpoint
            self.agent.setup()
            print("✓ Agent setup complete with RAG tools")

        except Exception as e:
            print(f"✗ Failed to setup agent: {e}")
            raise

    def chat(self, user_input: str) -> Dict[str, Any]:
        """
        Process a user message and generate a response using the agent.
        The agent will use available tools (RAG, search, etc.) to formulate responses.

        Args:
            user_input (str): The user's message or query

        Returns:
            dict: Response object with agent output, status, and metadata
        """
        # Check if agent is initialized before processing
        if not self.agent:
            return {
                "status": "error",
                "message": "Agent not initialized. Call setup_agent() first."
            }

        try:
            # Record user message in conversation history with timestamp
            self.conversation_history.append({
                "role": "user",
                "content": user_input,
                "timestamp": datetime.now().isoformat()
            })

            print(f"\n📝 User: {user_input}")
            print("-" * 60)

            # Execute the agent to process the user input and generate response
            response = self.agent.run(user_input)

            # Record agent response in conversation history
            response_text = str(response)
            self.conversation_history.append({
                "role": "assistant",
                "content": response_text,
                "timestamp": datetime.now().isoformat()
            })

            # Return successful response with metadata
            return {
                "status": "success",
                "user_input": user_input,
                "response": response,
                "turn": len(self.conversation_history) // 2
            }

        except Exception as e:
            print(f"✗ Error processing message: {e}")
            return {
                "status": "error",
                "message": str(e),
                "user_input": user_input
            }

    def get_conversation_history(self) -> List[Dict[str, Any]]:
        """
        Retrieve the complete conversation history.

        Returns:
            List[Dict[str, Any]]: List of all user and assistant messages with timestamps
        """
        return self.conversation_history

    def clear_history(self):
        """Clear the conversation history for starting a fresh session."""
        self.conversation_history = []
        print("✓ Conversation history cleared")


In [10]:
# ============================================================================
# EXAMPLE USAGE - Demonstration functions
# ============================================================================

def main():
    """
    Demonstrate the RAG chatbot functionality with sample queries.
    This function shows how to initialize, configure, and use the chatbot.
    """
    print("=" * 70)
    print("Oracle ADK RAG Chatbot Demo")
    print("=" * 70)

    # Configuration parameters - Replace with your actual values
    # Get the agent endpoint ID from your Oracle ADK deployment
    AGENT_ENDPOINT_ID = "ocid1.genaiagentendpoint.oc1.us-chicago-1.amaaaaaaknuwtjiadfdoluzq45s4owu7wvvwgtdtwvsau7h4ef75edqbmu5a"
    # OCI region where the agent is deployed
    REGION = "us-chicago-1"
    # Authentication method (api_key, instance_principal, resource_principal)
    AUTH_TYPE = "api_key"
    # OCI configuration profile to use from ~/.oci/config
    PROFILE = "DEFAULT_CHICAGO"

    # Step 1: Initialize the chatbot with OCI configuration
    print("\n1. Initializing RAG Chatbot...")
    print("-" * 70)

    try:
        chatbot = RAGChatbot(
            agent_endpoint_id=AGENT_ENDPOINT_ID,
            auth_type=AUTH_TYPE,
            profile=PROFILE,
            region=REGION
        )

        # Step 2: Setup the agent with RAG tools and instructions
        print("\n2. Setting up Agent with RAG Tools...")
        print("-" * 70)
        chatbot.setup_agent()

        # Step 3: Run sample queries through the chatbot
        print("\n3. Conducting Conversations...")
        print("-" * 70)

        # Sample queries to demonstrate the chatbot's capabilities
        sample_queries = [
            "What is Database Sharding?",
            "How does RAG work?",
            "Tell me about OCI Database services"
        ]

        # Process each sample query
        for query in sample_queries:
            response = chatbot.chat(query)

            # Display response or error
            if response["status"] == "success":
                print(f"\n🤖 Agent Response:")
                response["response"].pretty_print()
            else:
                print(f"\n❌ Error: {response['message']}")

        # Step 4: Display summary of conversation history
        print("\n4. Conversation History Summary")
        print("-" * 70)
        history = chatbot.get_conversation_history()
        print(f"Total exchanges: {len(history) // 2}")
        for i, msg in enumerate(history):
            role_icon = "👤" if msg["role"] == "user" else "🤖"
            print(f"{role_icon} {msg['role'].upper()}: {msg['content'][:60]}...")

    except Exception as e:
        print(f"\n❌ Failed to run chatbot: {e}")
        print("\nTroubleshooting:")
        print("1. Ensure your OCI config is set up: ~/.oci/config")
        print("2. Update AGENT_ENDPOINT_ID with your actual endpoint OCID")
        print("3. Verify your OCI region is correct")
        print("4. Check your authentication credentials")


def interactive_chat():
    """
    Run an interactive chatbot session allowing user input until exit.
    Provides a command-line interface for conversing with the RAG chatbot.
    """
    print("=" * 70)
    print("Interactive RAG Chatbot")
    print("=" * 70)
    print("Type 'exit' to quit, 'history' to see conversation history")
    print()

    # Agent endpoint ID - Replace with your actual endpoint
    AGENT_ENDPOINT_ID = 'ocid1.genaiagentendpoint.oc1.us-chicago-1.amaaaaaaknuwtjiadfdoluzq45s4owu7wvvwgtdtwvsau7h4ef75edqbmu5a'

    try:
        # Initialize chatbot with configuration
        chatbot = RAGChatbot(
            agent_endpoint_id=AGENT_ENDPOINT_ID,
            auth_type="api_key",
            profile="DEFAULT_CHICAGO",
            region="us-chicago-1"
        )

        # Setup agent with tools
        chatbot.setup_agent()

        # Interactive loop - continue until user exits
        while True:
            # Get user input from command line
            user_input = input("\n👤 You: ").strip()

            # Skip empty inputs
            if not user_input:
                continue

            # Exit the chatbot
            if user_input.lower() == "exit":
                print("\n👋 Goodbye!")
                break

            # Display conversation history
            if user_input.lower() == "history":
                history = chatbot.get_conversation_history()
                print("\n📋 Conversation History:")
                for msg in history:
                    print(f"  {msg['role']}: {msg['content'][:50]}...")
                continue

            # Process user query and display response
            response = chatbot.chat(user_input)
            if response["status"] == "success":
                print(f"\n🤖 Assistant: {response['response']}")
            else:
                print(f"\n❌ Error: {response['message']}")

    except Exception as e:
        print(f"\n❌ Chatbot error: {e}")


if __name__ == "__main__":
    # Apply nest_asyncio to allow nested event loops, which is common in environments like Colab
    #nest_asyncio.apply()
    # Run the demo with sample queries
    main()

    # Uncomment the line below to run interactive chat instead
    #interactive_chat()

Oracle ADK RAG Chatbot Demo

1. Initializing RAG Chatbot...
----------------------------------------------------------------------
✓ Agent client initialized
  - Region: us-chicago-1
  - Auth Type: api_key

2. Setting up Agent with RAG Tools...
----------------------------------------------------------------------


[01/28/26 13:51:41]  INFO     Checking integrity of agent details...

[01/28/26 13:51:42]  INFO     Checking synchronization of local and remote agent settings...

[01/28/26 13:51:43]  INFO     Checking synchronization of local and remote function tools...

╭───────────────────────────────── Local and remote function tools found ─────────────────────────────────╮
│ Local function tools (4):                                                                               │
│ ['get_conversation_context', 'get_document_content', 'retrieve_documents', 'search_oci_knowledge_base'] │
│                                                                                                         │
│ Remote function tools (4):                                                                              │
│ ['get_conversation_context', 'get_document_content', 'retrieve_documents', 'search_oci_knowledge_base'] │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────── Chat request to remote agent: None ───────────────────────────────────────╮
│ (Local --> Remote)                                                                                              │
│                                                                                                                 │
│ user message:                                                                                                   │
│ null                                                                                                            │
│                                                                                                                 │
│ performed actions by client:                                                                                    │
│ [{'action_id': '965f33f1-3641-4912-8634-4623902f4940', 'performed_action_type':                                 │
│ 'FUNCTION_CALLING_PERFORMED_ACTION', 'function_call_output': '{"status": "not_found", "message": "No documents  │
│ found for query: \'Autonomous Database\'", "documents": []}'}]                                                  │
│                                                                                                                 │
│ session id:                                                                                                     │
│ ocid1.genaiagentsession.oc1.us-chicago-1.amaaaaaa7mjirbaa2r7svbkdk4hvepwiairkcsgsrhmdom6xbvznvvch4dqq           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── Chat response from remote agent ────────────────────────────────────────╮
│ (Local <-- Remote)                                                                                              │
│                                                                                                                 │
│ agent message:                                                                                                  │
│ {                                                                                                               │
│     "role": "AGENT",                                                                                            │
│     "content": {                                                                                                │
│         "text": "Oracle Cloud Infrastructure (OCI) offers a range of database services, including Autonomous    │
│ Database, MySQL Database, PostgreSQL Database, NoSQL Database, and Database Backup. While I couldn't find       │
│ specific documents about these services, you can refer to the Oracle Cloud Infrastructure documentation and     │
│ knowledge base for more general information. Autonomous Database is a fully managed database service that       │
│ automates many administrative tasks, providing a high level of performance, security, and availability. If you  │
│ have any more specific questions or need further assistance, feel free to ask.",                                │
│         "citations": null,                                                                                      │
│         "paragraph_citations": null                                                                             │
│     },                                                                                                          │
│     "time_created": "2026-01-28T13:53:15.138000+00:00"                                                          │
│ }                                                                                                               │
│                                                                                                                 │
│ required actions for client to take:                                                                            │
│ null                                                                                                            │
│                                                                                                                 │
│ guardrail result:                                                                                               │
│ None                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


🤖 Agent Response:


╭────────────────────────────────────────────── Agent run response ───────────────────────────────────────────────╮
│ agent text message:                                                                                             │
│ Oracle Cloud Infrastructure (OCI) offers a range of database services, including Autonomous Database, MySQL     │
│ Database, PostgreSQL Database, NoSQL Database, and Database Backup. While I couldn't find specific documents    │
│ about these services, you can refer to the Oracle Cloud Infrastructure documentation and knowledge base for     │
│ more general information. Autonomous Database is a fully managed database service that automates many           │
│ administrative tasks, providing a high level of performance, security, and availability. If you have any more   │
│ specific questions or need further assistance, feel free to ask.                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


4. Conversation History Summary
----------------------------------------------------------------------
Total exchanges: 3
👤 USER: What is Database Sharding?...
🤖 ASSISTANT: raw_responses=[RawResponse(raw_data={'message': None, 'trace...
👤 USER: How does RAG work?...
🤖 ASSISTANT: raw_responses=[RawResponse(raw_data={'message': None, 'trace...
👤 USER: Tell me about OCI Database services...
🤖 ASSISTANT: raw_responses=[RawResponse(raw_data={'message': None, 'trace...
